In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_score, recall_score
import warnings
warnings.simplefilter (action = 'ignore')
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

df = pd.read_csv('/kaggle/input/hr-analytics-predictive-analysis/general_data.csv') #dataseti yükleme

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.set_option('display.max_columns', None)  #Tüm sütunları noktalama olmadan görmeyi sağlıyor
df

In [ ]:
df.info() #dataseti tanıma

In [ ]:
df.duplicated().sum()

In [ ]:
df['EducationField'].unique()

In [ ]:
missing_values = df.isnull().sum()
print("Eksik veriler:\n", missing_values)

In [ ]:
df['NumCompaniesWorked'].isnull().sum()

In [ ]:
df.dropna(subset=['NumCompaniesWorked', 'TotalWorkingYears'], inplace=True) #eksik veri sayısı oldukça az olduğundan ve modelin performansını etkilemeceğinden eksik değerli satırlar silindi.
print("Kalan veri sayısı:", len(df))

In [ ]:
# Sayısal sütunları seçelim
df_corr = df[['Age', 'DistanceFromHome', 'MonthlyIncome', 'NumCompaniesWorked', 'PercentSalaryHike', 'TotalWorkingYears', 'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager']]

# Korelasyon matrisini hesaplayalım
df_corr = df_corr.corr()

# Korelasyon matrisini görselleştirme
plt.figure(figsize=(10, 8))
sns.heatmap(data=df_corr, annot=True, cmap='BuGn')
plt.title('Korelasyon Matrisi')
plt.show()

Bu korelasyon matrisi, veri setindeki sayısal değişkenler arasındaki ilişkileri gösteriyor. Korelasyon katsayıları -1 ile 1 arasında değişir ve 1'e yakın değerler iki değişken arasında güçlü bir pozitif ilişki olduğunu, -1'e yakın değerler ise güçlü bir negatif ilişki olduğunu belirtir.

Grafikte dikkat çeken bazı noktalar şunlardır:

TotalWorkingYears ile Age arasında 0.68'lik güçlü bir pozitif korelasyon var, bu da beklenen bir durum çünkü çalışanların toplam çalışma yılları genellikle yaşla birlikte artar.
YearsWithCurrManager ve YearsAtCompany arasında 0.77'lik bir korelasyon görülüyor, bu da çalışanların genellikle aynı yöneticide uzun süre kaldıklarını gösteriyor.
YearsAtCompany ve TotalWorkingYears arasında da 0.63'lük bir korelasyon var, bu da şirkette geçirilen süre ile toplam çalışma yılları arasında anlamlı bir ilişki olduğunu gösteriyor.

In [ ]:
plt.figure(figsize=(12, 9))

# Kategorik değişkenler
categorical_cols = ['Gender', 'Department', 'JobRole', 'MaritalStatus', 'BusinessTravel']

for i, col in enumerate(categorical_cols):
    plt.subplot(3, 2, i + 1)
    x = df[col].value_counts().reset_index()
    x.columns = [col, 'count']  # Sütun isimlerini doğru ayarlıyoruz
    plt.title(col)
    plt.pie(x['count'], labels=x[col], autopct="%0.1f%%", colors=sns.color_palette('muted'))

plt.tight_layout()
plt.show()

Bu grafikler, cinsiyet (Gender), departman (Department), iş rolü (JobRole), medeni durum (MaritalStatus) ve iş seyahati (BusinessTravel) gibi kategorik değişkenlerin dağılımlarını gösteriyor.

1. Gender (Cinsiyet): Çalışanların %59.9'u erkek, %40.1'i ise kadın. Erkek çalışanların sayısının kadınlardan fazla olduğu görülüyor.

2. Department (Departman): Çalışanların %65.4'ü Araştırma ve Geliştirme (Research & Development) departmanında çalışırken, %30.4'ü Satış (Sales) departmanında yer alıyor. İnsan Kaynakları (Human Resources) departmanında çalışan oranı ise %4.3 ile en düşük.

3. JobRole (İş Rolü): Çalışanların en büyük kısmı %22.3 ile Satış Yöneticisi (Sales Executive) pozisyonunda. Bunu %19.9 ile Araştırma Bilimcisi (Research Scientist) ve %17.6 ile Laboratuvar Teknisyeni (Laboratory Technician) izliyor. En az çalışan %3.9 oranıyla Sağlık Temsilcisi (Healthcare Representative) rolünde.

4. MaritalStatus (Medeni Durum): Çalışanların %45.8'i evli (Married), %32.1'i bekar (Single), %22.1'i ise boşanmış (Divorced). Bu dağılımda evli çalışanların çoğunlukta olduğu görülüyor.

5. BusinessTravel (İş Seyahati): Çalışanların %70.9'u nadiren iş seyahati yaparken (Travel_Rarely), %18.8'i sık seyahat ediyor (Travel_Frequently). Seyahat etmeyenlerin (Non-Travel) oranı ise %10.2 ile en düşük.

In [ ]:
# Sayısal değişkenler için histogramlar
numerical_cols = ['Age', 'MonthlyIncome', 'YearsAtCompany']

# Histogramlar için görselleştirme
plt.figure(figsize=(16, 12))

for i, col in enumerate(numerical_cols):
    plt.subplot(2, 2, i + 1)
    sns.histplot(data=df, x=col, kde=True, bins=30, color='blue')
    plt.title(f"{col} Dağılımı")

plt.tight_layout()
plt.show()

Bu üç grafikte, yaş (Age), aylık gelir (MonthlyIncome) ve şirkette geçirilen yıl (YearsAtCompany) dağılımları incelenmiştir.

1. Yaş Dağılımı: Çalışanların yaş dağılımı genel olarak simetrik bir dağılım göstermektedir. 30'lu yaşlarda çalışan yoğunluğu en yüksektir, özellikle 35 yaş civarında zirve yapmaktadır. Bu, şirketin çalışan kitlesinin çoğunlukla orta yaş grubunda olduğunu göstermektedir.

2. Aylık Gelir Dağılımı: Aylık gelir (MonthlyIncome) dağılımında sağa çarpık bir yapı gözlemlenmektedir. Çalışanların büyük bir çoğunluğu 5000 ile 10.000 arasındaki maaş diliminde toplanmıştır. Ancak, gelir dağılımı geniş bir aralığa sahiptir ve 200.000'e kadar çıkan maaşlar da bulunmaktadır. Yüksek maaş alan çalışanların sayısı oldukça azdır ve bu, maaşlar arasındaki farklılıkların geniş olduğunu gösterir.

3. Şirkette Geçirilen Yıl Dağılımı (YearsAtCompany): Çalışanların şirkette çalışma süresi dağılımı daha çok düşük yıllar tarafına yoğunlaşmıştır. Çoğu çalışan 5 yıldan az bir süredir şirkette çalışmaktadır. Bunun yanında, 10 yılın üzerinde çalışan sayısı oldukça azdır. Bu durum, çalışanların şirkette uzun süre kalmadığını veya yeni çalışanların yoğunlukta olduğunu gösterebilir.

In [ ]:
# İşten ayrılma (Attrition) durumuna göre gelir dağılımı
plt.figure(figsize=(12, 6))
sns.boxplot(x='Attrition', y='MonthlyIncome', data=df)
plt.title("İşten Ayrılma Durumuna Göre Aylık Gelir Dağılımı")
plt.show()

Bu grafikte, işten ayrılma durumu (Attrition) ile aylık gelir (MonthlyIncome) arasındaki ilişkiyi incelemekteyiz. Görüldüğü üzere, işten ayrılmayan çalışanlar ("No") için aylık gelir dağılımı daha geniş bir aralıkta yer alıyor ve genel olarak daha yüksek seviyelerde bulunuyor. Öte yandan, işten ayrılan çalışanlar ("Yes") için aylık gelir genellikle daha düşük seviyelerde toplanmış durumda. Ayrıca, işten ayrılanlar grubunda bazı aşırı uç (outlier) değerler bulunuyor, ancak genel olarak bu gruptaki çalışanlar daha düşük maaş alıyorlar. Bu durum, düşük gelir seviyesinin çalışanların işten ayrılma olasılığını artırabileceğini düşündürebilir.

In [ ]:
# Attrition ve MonthlyIncome arasındaki ilişkiyi scatter plot ile görselleştirelim
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x="MonthlyIncome", y="Age", hue="Attrition", palette=sns.color_palette("pastel"))
plt.title("İşten Ayrılma Durumu ve Yaş-Gelir İlişkisi")
plt.show()

Bu grafikte, çalışanların yaş (Age), aylık gelir (MonthlyIncome), ve işten ayrılma durumu (Attrition) arasındaki ilişkiyi scatter plot ile incelemekteyiz. Grafikte, farklı yaş gruplarındaki çalışanların gelirlerine göre dağılımı ve işten ayrılma durumları gösterilmiştir. Mavi noktalar işten ayrılmayanları, turuncu noktalar ise işten ayrılanları temsil etmektedir. Genel olarak, aylık geliri düşük olan çalışanlar arasında işten ayrılma oranının daha yüksek olduğunu gözlemleyebiliyoruz. Ayrıca, yaş ilerledikçe, daha yüksek maaşlara sahip çalışanların işten ayrılma eğilimlerinin azaldığı söylenebilir. Yüksek maaş alan çalışanlar arasında ise işten ayrılma oranı oldukça düşük. Bu durum, daha yüksek maaşların çalışanların şirkette kalma olasılığını artırdığına işaret ediyor olabilir.

In [ ]:
# Departmanlara göre cinsiyet ve işten ayrılma oranlarını bar plot ile karşılaştıralım
plt.figure(figsize=(12, 6))
sns.countplot(data=df, x="Department", hue="Attrition", palette=sns.color_palette("pastel"))
plt.title("Departmanlara Göre İşten Ayrılma Oranı")
plt.show()

Bu grafikte, departmanlara göre işten ayrılma oranları (Attrition) karşılaştırılmaktadır. Görüldüğü üzere, Research & Development departmanında çalışanların büyük bir kısmı işte kalmayı tercih etmişken, Sales departmanında işten ayrılma oranı daha yüksek çıkmıştır. Özellikle satış departmanında işten ayrılma oranı diğer departmanlara kıyasla oldukça yüksektir. Öte yandan, Human Resources (İnsan Kaynakları) departmanında hem işte kalan hem de işten ayrılanların sayısı oldukça düşük seviyededir. Bu sonuçlar, departmanlar arasındaki işten ayrılma eğilimlerinin farklı olduğunu ve özellikle satış departmanında çalışanların işten ayrılma eğiliminin daha yüksek olduğunu göstermektedir.

In [ ]:
# Gruplama ve ortalama alma
grouped_df = df.groupby("Education")[["YearsAtCompany", "MonthlyIncome"]].mean()

# Yeni bir figür ve eksen oluştur
fig, ax1 = plt.subplots(figsize=(10, 6))

# İlk eksende YearsAtCompany'yi çiz
ax1.set_xlabel('Eğitim Seviyesi')
ax1.set_ylabel('Şirkette Geçen Yıl', color='blue')
grouped_df["YearsAtCompany"].plot(kind="bar", ax=ax1, color='blue', position=1, width=0.4)
ax1.tick_params(axis='y', labelcolor='blue')

# İkinci bir eksen oluştur ve MonthlyIncome'yi çiz
ax2 = ax1.twinx()
ax2.set_ylabel('Aylık Gelir', color='green')
grouped_df["MonthlyIncome"].plot(kind="bar", ax=ax2, color='green', position=0, width=0.4)
ax2.tick_params(axis='y', labelcolor='green')

plt.title("Eğitim Seviyesine Göre Şirkette Geçirilen Yıllar ve Aylık Gelir Ortalaması")
plt.show()

Bu grafikte, çalışanların eğitim seviyesine göre şirkette geçirdikleri yıllar (mavi çubuklarla gösterilen) ve aylık gelir ortalamaları (yeşil çubuklarla gösterilen) karşılaştırılmaktadır. Genel olarak, eğitim seviyesi arttıkça çalışanların hem şirkette geçirdikleri yılların hem de aldıkları aylık gelirin arttığı gözlemlenmektedir. Özellikle, eğitim seviyesi 5 (muhtemelen yüksek lisans veya doktora) olan çalışanlar en yüksek aylık gelir ortalamasına sahipken, daha düşük eğitim seviyelerinde maaşlar nispeten daha düşük seviyede kalmaktadır. Benzer şekilde, daha yüksek eğitim seviyesine sahip olan çalışanlar şirkette daha uzun süre çalışmışlardır, bu da deneyim ve kıdemle birleşen bir gelir artışı olduğunu göstermektedir.

In [ ]:
# Çalışanların işte geçirdikleri yıl, terfi süresi ve işten ayrılma arasındaki ilişki
plt.figure(figsize=(12, 8))
sns.boxplot(data=df, x="Attrition", y="YearsSinceLastPromotion", palette=sns.color_palette("Set2"))
plt.title("İşten Ayrılma Durumu ve Terfi Arasındaki İlişki")
plt.show()

Bu grafikte, işten ayrılma durumu ile son terfi arasındaki yıllar arasındaki ilişkiyi görüyoruz. İşten ayrılmayan çalışanların terfi süreleri daha geniş bir aralıkta dağılırken, ortalama terfi süresi 2-4 yıl arasında yoğunlaşmıştır ve birçok çalışan uzun süre terfi almadan çalışmıştır. Öte yandan, işten ayrılan çalışanların terfi süreleri daha kısa ve genellikle 2 yılın altındadır. Ayrıca, bu grupta aykırı değerlerin daha az olması, terfi alamayan çalışanların daha kısa süre içinde işten ayrılma eğiliminde olduğunu gösteriyor. Bu durum, çalışanların terfi almadıkça işten ayrılma olasılıklarının arttığını düşündürebilir.

In [ ]:
# Cinsiyet ve maaş arasındaki ilişkiyi gösterelim
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x="Gender", y="MonthlyIncome", palette="Blues")
plt.title("Cinsiyete Göre Maaş Dağılımı")
plt.show()

Bu grafikte, erkek ve kadın çalışanların aylık gelir dağılımı kıyaslanmaktadır. Görüldüğü üzere, erkek çalışanların maaşları genellikle kadınlara kıyasla daha yüksek bir ortalamaya sahiptir, çünkü erkeklerin medyan maaşı (kutunun ortasındaki çizgi) kadınlarınkinden daha yüksekte yer almaktadır. Ayrıca, erkek çalışanların maaşları daha geniş bir aralıkta dağılırken, kadın çalışanların maaşları daha dar bir dağılım sergilemektedir. Erkeklerde daha fazla uç (outlier) değer görülmektedir, bu da bazı erkek çalışanların maaşlarının diğerlerinden oldukça farklı olduğu anlamına gelir. Bu grafik, cinsiyetler arasında maaş farklarının belirgin olabileceğini göstermektedir.

In [ ]:
# Departman ve işten ayrılma durumu
df.groupby(['Department', 'Attrition'])['Attrition'].count()

Bu sonuçlar, farklı departmanlarda işten ayrılma oranlarının değişiklik gösterdiğini ortaya koyuyor. Human Resources departmanında işten ayrılma oranı oldukça yüksek (%30), bu da departmandaki çalışan memnuniyetiyle ilgili sorunlar olabileceğini düşündürebilir. Research & Development departmanında işten ayrılma oranı %15,6 iken, Sales departmanında bu oran %15. Genel olarak, işten ayrılmaların daha yüksek olduğu departman Human Resources departmanı diyebiliriz.

In [ ]:
# LabelEncoder nesnesi oluşturma
le = LabelEncoder()

# Dönüştürülecek kategorik sütunlar
columns_to_encode = ['MaritalStatus', 'Gender', 'Department', 'BusinessTravel', 'Attrition']

# Seçilen sütunlara Label Encoding uygulama
for col in columns_to_encode:
    df[col] = le.fit_transform(df[col])

print(df[columns_to_encode].head())

'MaritalStatus', 'Gender', 'Department', 'BusinessTravel', 'Attrition' sütunları az sayıda unique değer içerdiğinden bu sütunlara label encoding uygulandı.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# OneHotEncoder nesnesi oluşturma
ohe = OneHotEncoder()

# 'EducationField' ve 'JobRole' sütunlarına One-Hot Encoding uygulama
xd = ohe.fit_transform(df[['EducationField', 'JobRole']]).toarray()

# Sonucu DataFrame'e dönüştürme ve indeksleri sıfırlama
xd = pd.DataFrame(xd)
xd.columns = ohe.get_feature_names_out(['EducationField', 'JobRole'])
xd.reset_index(drop=True, inplace=True)

# Orijinal df'ye One-Hot Encoding sonucu oluşan sütunları ekleme, indeks sıfırlanmış olmalı
df.reset_index(drop=True, inplace=True)  # Orijinal df'nin indeksini de sıfırlıyoruz
df = pd.concat([df, xd], axis=1)

# Eski 'EducationField' ve 'JobRole' sütunlarını silme
df = df.drop(['EducationField', 'JobRole'], axis=1)

print(df.head())

'EducationField', 'JobRole' sütunları fazla sayıda eşsiz değer içerdiğinden one hot encoding uygulandı.

In [ ]:
df = df.drop(['Over18'], axis=1) 

Tek değerli değişkenlerin varyansı yoktur, bu da modelin bu değişkenden hiçbir bilgi öğrenemeyeceği anlamına gelir. Bu tür gereksiz sütunları kaldırmak, modelin gereksiz hesaplamalar yapmasını engeller ve veri setini daha verimli hale getirir.

In [ ]:
df

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Bağımlı değişken (Attrition)
y = df[['Attrition']]  # "Attrition" sütunu bağımlı değişken olarak alınır

# Bağımsız değişkenler (Attrition hariç tüm sütunlar)
x = df.drop(["Attrition"], axis=1)  # "Attrition" sütunu hariç diğer sütunlar bağımsız değişkenler olur


In [ ]:
# Veriyi eğitim ve test setlerine bölme (train %70, test %30)
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.70, random_state=35)

In [ ]:
# Veriyi ölçeklendirme (StandardScaler kullanarak)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

print(x_train[:5])  
print(y_train[:5])  

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.cluster import KMeans, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

In [ ]:
# Gözetimli öğrenme sınıflandırıcıları
supervised_classifiers = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest Classifier": RandomForestClassifier(),
    "Gradient Boosting Classifier": GradientBoostingClassifier(),
    "SVC": SVC(),
    "XGBoost Classifier": XGBClassifier()
}

# Gözetimsiz öğrenme modelleri
unsupervised_models = {
    "K-Means Clustering": KMeans(n_clusters=2),
    "DBSCAN": DBSCAN(),
    "Gaussian Mixture Model": GaussianMixture(n_components=2)
}

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Initialize a dictionary to store results
results = {}

# Drop the target 'Attrition' from the features for unsupervised models
X_unsupervised = df.drop(["Attrition"], axis=1)
y_true = df['Attrition'].replace({'Yes': 1, 'No': 0})  # Binary labels for Attrition

# 1. K-Means Clustering
kmeans = KMeans(n_clusters=2, random_state=42)

# Measure fit time
start_time = time.time()
kmeans_labels = kmeans.fit_predict(X_unsupervised)
fit_time = time.time() - start_time

# Measure accuracy, precision, recall, and F1 by comparing cluster labels with true labels
# You may need to align the cluster labels with the actual labels as K-Means doesn't guarantee label order
accuracy_kmeans = accuracy_score(y_true, kmeans_labels)
precision_kmeans = precision_score(y_true, kmeans_labels, average='binary')
recall_kmeans = recall_score(y_true, kmeans_labels, average='binary')
f1_kmeans = f1_score(y_true, kmeans_labels, average='binary')

# Save results
results['K-Means'] = {
    'fit_time': fit_time,
    'test_accuracy': accuracy_kmeans,
    'test_precision': precision_kmeans,
    'test_recall': recall_kmeans,
    'test_f1': f1_kmeans
}

# 2. DBSCAN
dbscan = DBSCAN()

# Measure fit time
start_time = time.time()
dbscan_labels = dbscan.fit_predict(X_unsupervised)
fit_time = time.time() - start_time

# Calculate metrics (if DBSCAN finds clusters, otherwise it may label most points as noise)
if len(set(dbscan_labels)) > 1:
    accuracy_dbscan = accuracy_score(y_true, dbscan_labels)
    precision_dbscan = precision_score(y_true, dbscan_labels, average='binary', zero_division=1)
    recall_dbscan = recall_score(y_true, dbscan_labels, average='binary', zero_division=1)
    f1_dbscan = f1_score(y_true, dbscan_labels, average='binary', zero_division=1)
else:
    accuracy_dbscan = precision_dbscan = recall_dbscan = f1_dbscan = 0

# Save results
results['DBSCAN'] = {
    'fit_time': fit_time,
    'test_accuracy': accuracy_dbscan,
    'test_precision': precision_dbscan,
    'test_recall': recall_dbscan,
    'test_f1': f1_dbscan
}

# 3. Gaussian Mixture Model (GMM)
gmm = GaussianMixture(n_components=2, random_state=42)

# Measure fit time
start_time = time.time()
gmm_labels = gmm.fit_predict(X_unsupervised)
fit_time = time.time() - start_time

# Evaluate metrics for GMM
accuracy_gmm = accuracy_score(y_true, gmm_labels)
precision_gmm = precision_score(y_true, gmm_labels, average='binary')
recall_gmm = recall_score(y_true, gmm_labels, average='binary')
f1_gmm = f1_score(y_true, gmm_labels, average='binary')

# Save results
results['GMM'] = {
    'fit_time': fit_time,
    'test_accuracy': accuracy_gmm,
    'test_precision': precision_gmm,
    'test_recall': recall_gmm,
    'test_f1': f1_gmm
}

# Display the results
import pandas as pd
results_df = pd.DataFrame(results)
print(results_df)

test_accuracy oldukça düşük olduğundan gözetimsiz öğrenme bu veriseti için uygun değildir

In [ ]:
# Gözetimli öğrenme modelleri için eğitim ve performans değerlendirmesi
for name, model in supervised_classifiers.items():
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    
    print(f"Model: {name}")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(f"Precision: {precision_score(y_test, y_pred):.4f}")
    print(f"Recall: {recall_score(y_test, y_pred):.4f}")
    print(f"F1 Score: {f1_score(y_test, y_pred):.4f}\n")
    
    # Çapraz doğrulama ile performans
    cv_scores = cross_validate(model, x_train, y_train, scoring=['accuracy', 'precision', 'recall', 'f1'], cv=10)
    cv_results = pd.DataFrame(cv_scores)
    print(f"{name} Cross-Validation Results:\n", cv_results.mean())

In [ ]:
# Gözetimli öğrenme modelleri için eğitim ve performans değerlendirmesi
for name, model in supervised_classifiers.items():
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)

    print(f"Model: {name}")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(f"Precision: {precision_score(y_test, y_pred):.4f}")
    print(f"Recall: {recall_score(y_test, y_pred):.4f}")
    print(f"F1 Score: {f1_score(y_test, y_pred):.4f}\n")
    
    # Çapraz doğrulama ile performans
    cv_scores = cross_validate(model, x_train, y_train, scoring=['accuracy', 'precision', 'recall', 'f1'], cv=10)
    cv_results = pd.DataFrame(cv_scores)
    print(f"{name} Cross-Validation Results:\n", cv_results.mean())
    print("\n############################################################################\n")

**XGBoost Classifier**, benim çalıştığım çalışan işten ayrılma (Attrition) tahmini probleminde kullanılan güçlü bir makine öğrenme algoritmasıdır. XGBoost, ardışık karar ağaçları oluşturarak sınıflandırma doğruluğunu iteratif olarak iyileştirir. Bu model, her adımda hataları minimize etmek için bir önceki modelin hatalarını düzelten yeni ağaçlar ekler. XGBoost’un öne çıkan özelliklerinden biri, yerleşik düzenleme (regularization) teknikleri sayesinde aşırı uyumu (overfitting) kontrol altında tutabilmesidir. Ayrıca, paralel işlem yeteneği sayesinde büyük veri setleri üzerinde hızlı sonuçlar üretir ve eksik verilerle çalışabilme yeteneği sunar.



**MODEL SEÇİMİ**

Projemde XGBoost Classifier, doğruluk, precision, recall ve f1 score metriklerinde en iyi performansı göstermiştir. Özellikle çalışanların işten ayrılma durumunu yüksek bir başarıyla tahmin edebilmiştir. %98.94 doğruluk oranı ve %100 precision ile XGBoost, sınıflandırma problemi için en uygun model olarak öne çıkmıştır. Bu modelin güçlü performansı ve esnekliği, işten ayrılma gibi kritik sınıflandırma problemlerinde etkili ve güvenilir tahminler sunar.

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

# Daha az CPU harcayan hiperparametre arama alanı
param_dist = {
    'n_estimators': [50, 100, 150], 
    'learning_rate': [0.001, 0.01, 0.05, 0.1, 0.2],  
    'max_depth': [3, 5, 7, 9, 12],  
    'subsample': [0.5, 0.6, 0.7, 0.8, 0.9],  
    'colsample_bytree': [0.7, 0.8], 
    'gamma': [0, 0.1],  
    'reg_lambda': [1, 10], 
    'reg_alpha': [0, 0.1]  
}
# XGBoost modeli
xgb = XGBClassifier(random_state=42)

# RandomizedSearchCV ile hiperparametre optimizasyonu
random_search = RandomizedSearchCV(estimator=xgb, param_distributions=param_dist, n_iter=50, 
                                   scoring='accuracy', cv=5, verbose=1, random_state=42, n_jobs=-1)
random_search.fit(x_train, y_train)

# En iyi parametreleri ve en iyi başarı oranını yazdırma
print("Best parameters: ", random_search.best_params_)
print("Best score (accuracy): ", random_search.best_score_)

XGBoost zaten genellikle optimize edilmiş varsayılan parametrelerle çalışır, bu nedenle performans varsayılan ayarlarla iyi olabilir. Bu sebepten ve modelin doğruluğunun çok yüksek (0.974893) olmasından dolayı doğruluğu daha da yükselten hiper parametreler ayarlanmadı.

In [ ]:
from sklearn.metrics import confusion_matrix, mean_squared_error, mean_absolute_error

# XGBoost modelinin en iyi parametrelerle eğitilmiş halini kullanarak tahmin yapma
y_pred = random_search.best_estimator_.predict(x_test)

# Karışıklık matrisi oluşturma
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# Mean Squared Error (MSE) hesaplama
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse:.4f}")

# Mean Absolute Error (MAE) hesaplama
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae:.4f}")

**Confusion Matrix (Karışıklık Matrisi)**:

Bu matrise göre, model 1131 doğru negatif sınıf tahmini (Attrition = No) ve 172 doğru pozitif sınıf tahmini (Attrition = Yes) yapmıştır.
Sadece 12 yanlış negatif tahmin yapılmış; yani model bu 12 kişiyi işten ayrılmayacakmış gibi tahmin etse de, aslında işten ayrılmışlardır.
Yanlış pozitif tahmin yok; yani model hiç kimseyi yanlış bir şekilde işten ayrılacakmış gibi sınıflandırmamış.
Bu durumda, model hem pozitif sınıf (işten ayrılma) hem de negatif sınıf (işte kalma) için oldukça doğru tahminler yapıyor. Yanlış negatiflerin sayısı çok düşük olduğundan, modelin işten ayrılmaları yakalama yeteneği (recall) oldukça iyi gözüküyor.

**Mean Squared Error (MSE)**: 0.0091, yani modelin tahminleriyle gerçek değerler arasındaki kare farkların ortalaması oldukça düşük.

**Mean Absolute Error (MAE)**: 0.0091, yani modelin tahminleriyle gerçek değerler arasındaki mutlak farkların ortalaması da oldukça düşük.